## Librerías

In [1]:
import ee
import json

## Autenticación

Primeros autenticarnos con nuestras credenciales, para esto necesitamos registrarnos.

In [ ]:
ee.Authenticate()

## Inicialización de la librería

In [3]:
ee.Initialize()

## Delimitación de la zona/zonas de estudio

Para esto vamos a obtener la geometría desde un archivo geojson.

In [6]:
with open(r'C:\Users\DELL\PycharmProjects\Tesis\Parcelas\Parcelas_estudio.geojson', encoding='utf-8') as geom:
    rancho = json.load(geom)

In [ ]:
rancho

## Pasamos este json a un FeatureCollection

Esto para poder operar las geometrías con las colecciones de imágenes.

In [8]:
fc = ee.FeatureCollection(rancho)

In [52]:
fc_bbox = fc.geometry().bounds()

Vamos a calcular el área para ver que coincida con las dimensiones esperadas.

In [ ]:
for feature in fc.getInfo()['features']:
    fn = ee.Feature(feature)
    area = fn.geometry().area()
    print(f'el area es {area.divide(10000).getInfo()} ha')
    print(fn.getInfo()['properties'])

# Colección de imágenes

## Misión Sentinel 2

La colección es de las imágenes con nivel de procesamiento 2A.

In [88]:
start_year = '2022-01-01'
end_year = '2023-07-01'

In [89]:
ic = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED").filterDate(start=start_year, opt_end=end_year).filter(ee.Filter.eq('MGRS_TILE', '14QML'))

In [90]:
ic.size().getInfo()

226

En la celda anterior se definió la colección de imágenes con un rango de fechas determinado y que solo sean las que coinciden con el **TILE_ID** = 14QML, que abarca la región de interés.

### AOI dentro de la imagen

Vamos a asegurarnos que el área de interés se encuentre dentro de la imagen, para esto se comparan las geometrías de ambos elementos.

In [60]:
def does_intersect(img, feature_geom):
    
    intersecta = img.geometry().intersects(feature_geom, maxError=0.01)
    
    return intersecta.getInfo()


### Filtro de nubes

En este apartado vamos a filtrar las imágenes que tienen nubes en la zona de estudio, las que presenten nubosidad en el área serán descartadas.

In [24]:
def count_cloud_pixels(img, cloud_percent, aoi):
    
    cloud_band = img.select('MSK_CLDPRB').gt(cloud_percent)
    pixel_count = cloud_band.reduceRegion(
        reducer = ee.Reducer.frequencyHistogram(),
        geometry = aoi,
        scale = 10,
        maxPixels = 1e9
    )
    
    return pixel_count.get('MSK_CLDPRB').getInfo()

## Exportación de imágenes a Google Drive

Teniendo los **id's** procedemos a exportar las imágenes a la carpeta de nuestra preferencia y con los parámetros deseados.

In [72]:
def export_images(folder: str, img, bands, aoi, to_crs: str, filename):
    
    raw_image = img.select(bands)
    
    export_params = {
        'driveFolder' : f'{folder}',
        'fileFormat': 'GeoTiff',
        'scale': 10,
        'region': aoi,
        'crs': to_crs,
    }
    
    task = ee.batch.Export.image.toDrive(image=raw_image, description=filename, **export_params)
    
    task.start()
    task.status()
    
    return print(f"{filename} exportada correctamente.")

In [ ]:
inside = 0
out = 0
cloud_free = 0

for image in ic.getInfo()['features']:
    resultado = does_intersect(img=ee.Image(image['id']), feature_geom=fc_bbox)
    #print(resultado)
    if resultado:
        counted_pixels = count_cloud_pixels(img=ee.Image(image['id']), cloud_percent=50, aoi=fc)
        print(counted_pixels)
        if '1' not in counted_pixels:
            for feature in fc.getInfo()['features']:
                parcela_feature = ee.Feature(feature)
                #print(f"La imagen {image['id']} intersecta")
                #print(f"{image['id']} está completamente libre de nubes")
                #print(f"Parcela {parcela_feature.getInfo()['properties']['Id']} con la imagen {image['id']} con la geometria {parcela_feature.geometry().getInfo()}")
                export_images(folder='Tesis_caña2', img=ee.Image(image['id']),bands=['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B11', 'B12'], aoi=parcela_feature.geometry(), to_crs='EPSG:32614', filename=f"Parcela_{parcela_feature.getInfo()['properties']['Id']}_{image['id'].replace('/', '_')}")
            cloud_free += 1
        #free_pixels, cloud_pixels = counted_pixels['0'], counted_pixels['1']
        #print(free_pixels, cloud_pixels)
        inside += 1
    else:
        #print(f"La imagen {image['id']} no coincide")
        out += 1

        
print(f'Dentro: {inside}, No coincide: {out}, Libres de nubes: {cloud_free}')
